# Direct Linear Solvers

This notebook covers direct methods for solving systems of linear equations of the form $Ax = b$ and matrix factorizations.

## Methods
1. **Gaussian Elimination**: Reduces matrix to row echelon form.
2. **LU Decomposition**: Factors matrix $A$ into lower triangular $L$ and upper triangular $U$ such that $A=LU$.
3. **Cholesky Decomposition**: Factors geometric positive-definite matrix $A$ into $LL^T$.

In [1]:
import numpy as np
import numpy.linalg as lng

## 1. Gaussian Elimination
Basic implementation with back substitution.

In [2]:
def gaussian_elimination(A, b):
    """
    Solves Ax = b using Gaussian Elimination with back substitution.
    """
    n = len(b)
    # Augmented matrix
    Ab = np.hstack([A, b.reshape(-1, 1)]).astype(float)
    
    # Forward Elimination
    for i in range(n):
        # Pivot
        if Ab[i, i] == 0:
            raise ValueError("Zero pivot encountered. Pivoting not implemented.")
            
        for j in range(i + 1, n):
            factor = Ab[j, i] / Ab[i, i]
            Ab[j, i:] -= factor * Ab[i, i:]
            
    # Back Substitution
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        sum_ax = np.dot(Ab[i, i+1:n], x[i+1:n])
        x[i] = (Ab[i, n] - sum_ax) / Ab[i, i]
        
    return x

## 2. LU Decomposition
Decomposes $A$ into $L$ and $U$ such that $A = LU$.

In [3]:
def lu_decomposition(A):
    """
    Performs LU decomposition of A using Doolittle's algorithm (L has unit diagonal).
    Returns L, U.
    """
    n = A.shape[0]
    L = np.eye(n)
    U = np.zeros((n, n))
    
    for i in range(n):
        # Upper Triangular
        for k in range(i, n):
            sum_l_u = sum(L[i, j] * U[j, k] for j in range(i))
            U[i, k] = A[i, k] - sum_l_u
            
        # Lower Triangular
        for k in range(i + 1, n):
            sum_l_u = sum(L[k, j] * U[j, i] for j in range(i))
            L[k, i] = (A[k, i] - sum_l_u) / U[i, i]
            
    return L, U

## 3. Cholesky Decomposition
For Symmetric Positive Definite matrices, $A = LL^T$.

In [4]:
def cholesky_decomposition(A):
    """
    Performs Cholesky decomposition of symmetric positive-definite matrix A.
    Returns L such that A = L * L.T
    """
    n = A.shape[0]
    L = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i + 1):
            sum_k = sum(L[i, k] * L[j, k] for k in range(j))
            
            if i == j:
                val = A[i, i] - sum_k
                if val <= 0:
                    raise ValueError("Matrix is not positive definite")
                L[i, j] = np.sqrt(val)
            else:
                L[i, j] = (A[i, j] - sum_k) / L[j, j]
                
    return L

### Tests

In [5]:
# Test System
A = np.array([[4, 1, 1], [1, 4, 1], [1, 1, 4]], dtype=float)
b = np.array([12, 3, 7], dtype=float)

print("--- Gaussian Elimination ---")
print("Solution:", gaussian_elimination(A, b))

print("\n--- LU Decomposition ---")
L, U = lu_decomposition(A)
print("L:\n", L)
print("U:\n", U)
print("Reconstructed A:\n", np.dot(L, U))

print("\n--- Cholesky Decomposition ---")
try:
    L_chol = cholesky_decomposition(A)
    print("L:\n", L_chol)
    print("Reconstructed A (L*L.T):\n", np.dot(L_chol, L_chol.T))
except Exception as e:
    print(e)

--- Gaussian Elimination ---
Solution: [ 2.77777778 -0.22222222  1.11111111]

--- LU Decomposition ---
L:
 [[1.   0.   0.  ]
 [0.25 1.   0.  ]
 [0.25 0.2  1.  ]]
U:
 [[4.   1.   1.  ]
 [0.   3.75 0.75]
 [0.   0.   3.6 ]]
Reconstructed A:
 [[4. 1. 1.]
 [1. 4. 1.]
 [1. 1. 4.]]

--- Cholesky Decomposition ---
L:
 [[2.         0.         0.        ]
 [0.5        1.93649167 0.        ]
 [0.5        0.38729833 1.8973666 ]]
Reconstructed A (L*L.T):
 [[4. 1. 1.]
 [1. 4. 1.]
 [1. 1. 4.]]
